# Тестирование деревьев решений

In [34]:
import sys
import os
current_dir = os.path.abspath('')
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, project_root)

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import OneHotEncoder
from utils.metrics import cross_validation, create_model, accuracy, mse, cross_validation_with_pruning

## Тестирование дерева классификации

In [ ]:
from trees.ClassifierTree import DecisionTreeClassifier, prune
from sklearn.tree import DecisionTreeClassifier as EtalonClassifier

Подготовка данных:

In [10]:
data_classify=pd.read_csv('../data/weather_classification_data.csv', sep=',')

Для собственной реализации:

In [37]:
X_1=data_classify.iloc[:,0:10].to_numpy() 
Y_1=data_classify.iloc[:,10].to_numpy().reshape(-1)
X_t_1, X_val_1, y_t_1, y_val_1 = train_test_split(X_1, Y_1, test_size=0.1, random_state=24)
X_train_1, X_test_1,y_train_1, y_test_1 = train_test_split(X_t_1, y_t_1, test_size=0.33, random_state=24)

Для эталонной реализации:

In [38]:
df_c=data_classify.copy(deep=True)
ohe=OneHotEncoder(sparse_output=False)
X_2=ohe.fit_transform(df_c[['Cloud Cover','Season','Location']])
X=np.concatenate((X_2,df_c[['Temperature','Humidity','Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index','Visibility (km)' ]].to_numpy()),axis=1)
y=df_c[['Weather Type']].to_numpy().reshape(-1)
X_t, X_val, y_t, y_val = train_test_split(X, y, test_size=0.1, random_state=24)#Делим на точно такие же выборки
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, test_size=0.33, random_state=24) 

In [22]:
dec=DecisionTreeClassifier()
start = time.time()
dec.fit(X_train_1,y_train_1, crit='e') 
end = time.time()
print("Время выполнения собственной реализации (критерий энтропии):",
      (end-start) * 10**3, "ms")
dec=DecisionTreeClassifier()
start = time.time()
dec.fit(X_train_1,y_train_1, crit='e') 
end = time.time()
print("Время выполнения собственной реализации (критерий Донского):",
      (end-start) * 10**3, "ms")
etc=EtalonClassifier(criterion='entropy')
start = time.time()
etc.fit(X_train, y_train)
end=time.time()
print("Время выполнения эталонной реализации программы:",
      (end-start) * 10**3, "ms")

Время выполнения собственной реализации (критерий энтропии): 44918.75720024109 ms
Время выполнения собственной реализации (критерий Донского): 45183.78973007202 ms
Время выполнения эталонной реализации программы: 29.50906753540039 ms


In [39]:
print("Accuracy написанной вручную модели с использованием кросс-валидации (критерий энтропии):", np.round(cross_validation(DecisionTreeClassifier,X_1,Y_1,5, fit_params={'crit':'e'}),4))
print("Accuracy написанной вручную модели с использованием кросс-валидации (критерий Донского):", np.round(cross_validation(DecisionTreeClassifier,X_1,Y_1,5, fit_params={'crit':'d'}),4))

Accuracy написанной вручную модели с использованием кросс-валидации (критерий энтропии): 0.8862
Accuracy написанной вручную модели с использованием кросс-валидации (критерий Донского): 0.8873


Также был реализован алгоритм прунинга дерева классификации:

In [24]:
dec=DecisionTreeClassifier()
dec.fit(X_train_1,y_train_1, crit='e')
print('Accuracy до прунинга:', accuracy(dec.predict(X_test_1), y_test_1))
print('Узлы до прунинга:', dec.rules)
dec.rules=prune(dec, X_val_1,y_val_1)
print('Accuracy после прунинга:', accuracy(dec.predict(X_test_1), y_test_1))
print('Узлы после прунинга:',dec.rules)

Accuracy до прунинга: 0.9033409844427442
Узлы до прунинга: [[['0', 'node', 0, 9.0]], [['00', 'node', 7, 'Winter'], ['01', 'node', 8, 5.5]], [['000', 'node', 3, 30.0], ['001', 'node', 1, 60], ['010', 'node', 6, 4], ['011', 'node', 6, 5]], [['0000', 'node', 1, 71], ['0001', 'node', 6, 10], ['0010', 'node', 2, 11.0], ['0011', 'node', 8, 7.5], ['0100', 'node', 3, 50.0], ['0101', 'node', 4, 'overcast'], ['0110', 'node', 3, 54.0], ['0111', 'node', 3, 20.0]], [['00000', 'node', 3, 14.0], ['00001', 'node', 0, -2.0], ['00010', 'node', 0, -17.0], ['00011', 'node', 0, -17.0], ['00100', 'node', 6, 1], ['00101', 'node', 4, 'overcast'], ['00110', 'node', 5, 1006.2], ['00111', 'node', 0, -5.0], ['01000', 'node', 8, 5.0], ['01001', 'node', 5, 1020.65], ['01010', 'node', 2, 21.0], ['01011', 'node', 2, 11.0], ['01100', 'node', 8, 10.5], ['01101', 'node', 1, 76], ['01110', 'node', 8, 12.5], ['01111', 'node', 1, 79]], [['000000', 'node', 2, 4.0], ['000001', 'node', 5, 920.18], ['000010', 'node', 1, 79], [

Рассмотрим его точность:

In [25]:
print("Accuracy написанной вручную модели с прунингом:", np.round(cross_validation_with_pruning(create_model(DecisionTreeClassifier),X_t_1,y_t_1,X_val_1,y_val_1,5),4))
print("Accuracy эталонной реализации модели с использованием кросс-валидации :", np.round(cross_validation(create_model(EtalonClassifier,criterion='entropy'),X_t,y_t,5),4))

Accuracy написанной вручную модели с прунингом: 0.8821
Accuracy эталонной реализации модели с использованием кросс-валидации : 0.905


Точность классификации незначительно упала, однако дерево теперь имеет намного меньший размер

## Тестирование деревьев регрессии

Подготовка данных:

In [ ]:
from trees.RegressionTree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor as EtalonRegressor

In [29]:
data_regression=pd.read_csv('../data/insurance.csv', sep=',')#Загрузка данных для регрессии
X_2=data_regression[['age','sex','bmi','children', 'smoker','region']].to_numpy()
Y_2=data_regression[['charges']].to_numpy().reshape(-1)
X_train_2, X_test_2,y_train_2, y_test_2 = train_test_split(X_2, Y_2, test_size=0.33, random_state=24)

In [30]:
df_r=data_regression.copy(deep=True)
ohe2=OneHotEncoder(sparse_output=False)
X_2_e=ohe2.fit_transform(df_r[['sex','smoker','region']])
X=np.concatenate((X_2_e,df_r[['age','bmi','children']].to_numpy()),axis=1)
y=df_r[['charges']].to_numpy().reshape(-1)
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=24)

In [31]:
dtc=DecisionTreeRegressor(7)#Создание дерева с максимальной глубиной
start = time.time()
dtc.fit(X_train_2,y_train_2)#Обучение дерева
end = time.time()
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 4149.323225021362 ms


In [32]:
etr=EtalonRegressor(criterion='squared_error', max_depth=7)
start = time.time()
etr.fit(X_train, y_train)
end=time.time()
print("Время выполнения программы:",
      (end-start) * 10**3, "ms")

Время выполнения программы: 1.9984245300292969 ms


In [33]:
print("R^2 написанной вручную модели с использованием кросс-валидации:", np.round(cross_validation(create_model(DecisionTreeRegressor,depth=7),X_2,Y_2,5, regression=True),4))
print("R^2 эталонной реализации модели с использованием кросс-валидации :", np.round(cross_validation(create_model(EtalonRegressor,criterion='squared_error', max_depth=7),X,y,5, regression=True),4))

R^2 написанной вручную модели с использованием кросс-валидации: 0.8178
R^2 эталонной реализации модели с использованием кросс-валидации : 0.8052
